In [ ]:
# Proposal

## Title: Credit Score Classification 

https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data 


### Introduction:
A credit report is a summary of a person’s credit history and is created when you borrow money or apply for a credit card. A credit score is a 3 digit number calculated based on your credit report that summarizes how well you manage your credit and how risky it would be for someone to lend you money. The higher credit score the better your rating is.


A credit score is calculated based on a few different factors such as:
Annual_income

Monthly_Inhand_Salary

Num_Bank_Accounts

Num_Credit_Card

Interest_Rate

Num_of_Loan

Type_of_Loan

Delay_from_due_date

Num_of_Delayed_Payment

Changed_Credit_Limit

Num_Credit_Inquiries

Credit_Mix

Outstanding_Debt

Credit_Utilization_Ratio

Credit_History_Age

Payment_of_Min_Amount

Total_EMI_per_month

Amount_invested_monthly

Payment_Behaviour

Monthly_Balance

